In [5]:
print("ADITYA RAM S H\n1BM22CS019\n")
# Define the knowledge base
knowledge_base = {
    "facts": {
        "American(Robert)": True,
        "Missile(T1)": True,
        "Owns(A, T1)": True,
        "Enemy(A, America)": True,
        "Sells(Robert, T1, A)": True
    },
    "rules": [
        # Rule: Missile(x) => Weapon(x)
        {"if": ["Missile(x)"], "then": "Weapon(x)"},

        # Rule: Enemy(x, America) => Hostile(x)
        {"if": ["Enemy(x, America)"], "then": "Hostile(x)"},

        # Rule: American(p) ∧ Weapon(q) ∧ Sells(p, q, r) ∧ Hostile(r) => Criminal(p)
        {
            "if": ["American(p)", "Weapon(q)", "Sells(p, q, r)", "Hostile(r)"],
            "then": "Criminal(p)"
        }
    ]
}

# Forward chaining algorithm
def forward_chaining(kb):
    derived_facts = set(kb["facts"].keys())  # Start with the initial facts
    new_facts = True  # Flag to track if new facts are derived

    while new_facts:
        new_facts = False
        # Iterate over a COPY of derived_facts to avoid size change issues
        current_facts = derived_facts.copy()

        for rule in kb["rules"]:
            if_conditions = rule["if"]
            then_condition = rule["then"]

            # Check all combinations of facts and rules
            substitutions = try_match_conditions(if_conditions, current_facts)
            if substitutions:
                # Apply the substitutions to the "then" part of the rule
                derived_fact = apply_substitution(then_condition, substitutions)
                if derived_fact not in derived_facts:  # Add new fact if not already present
                    derived_facts.add(derived_fact)
                    kb["facts"][derived_fact] = True
                    new_facts = True  # New fact found, continue reasoning

    return derived_facts

def try_match_conditions(conditions, facts):
    """
    Try to match all conditions with the current set of facts.
    Returns substitutions if all conditions are satisfied, otherwise None.
    """
    substitutions = {}
    for condition in conditions:
        matched = False
        for fact in facts:
            sub = unify(condition, fact)
            if sub:
                substitutions.update(sub)
                print(sub)
                matched = True
                break
        if not matched:  # If one condition is not matched, return None
            return None
    return substitutions

def unify(condition, fact):
    """
    Check if a fact matches a condition and return variable substitutions.
    Example:
    unify("Missile(x)", "Missile(T1)") => {"x": "T1"}
    """
    if "(" in condition and "(" in fact:
        cond_pred, cond_args = parse_predicate(condition)
        fact_pred, fact_args = parse_predicate(fact)
        if cond_pred == fact_pred and len(cond_args) == len(fact_args):
            return {cond_args[i]: fact_args[i] for i in range(len(cond_args)) if cond_args[i].islower()}
    return None

def parse_predicate(statement):
    """
    Parse a predicate into its name and arguments.
    Example:
    "Missile(T1)" => ("Missile", ["T1"])
    """
    pred, args = statement.split("(")
    args = args.strip(")").split(",")
    return pred.strip(), [arg.strip() for arg in args]

def apply_substitution(statement, substitutions):
    """
    Apply substitutions to a statement.
    Example:
    apply_substitution("Weapon(x)", {"x": "T1"}) => "Weapon(T1)"
    """
    pred, args = parse_predicate(statement)
    substituted_args = [substitutions.get(arg, arg) for arg in args]
    return f"{pred}({', '.join(substituted_args)})"

# Run forward chaining
result = forward_chaining(knowledge_base)

# Check if Criminal(Robert) is derived
if "Criminal(Robert)" in result:
    print("\nQuery Proven: Criminal(Robert) is true.")
else:
    print("\nQuery Not Proven: Criminal(Robert) is not true.")



ADITYA RAM S H
1BM22CS019

{'x': 'T1'}
{'x': 'A'}
{'p': 'Robert'}
{'x': 'T1'}
{'x': 'A'}
{'p': 'Robert'}
{'q': 'T1'}
{'p': 'Robert', 'q': 'T1', 'r': 'A'}
{'r': 'A'}
{'x': 'T1'}
{'x': 'A'}
{'p': 'Robert'}
{'q': 'T1'}
{'p': 'Robert', 'q': 'T1', 'r': 'A'}
{'r': 'A'}

Query Proven: Criminal(Robert) is true.
